# Extract ObjectTables Results in  OGA

 work with Weakly_2023_44
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/21
- update : 2023/12/22


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

## Configuration

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
fileobj_out="objectTable_202312.csv"
filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

## Give the collection

In [ ]:
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"

## Config

# object

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType='objectTable', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
all_ref_for_src = []
all_objectsTables = []

for i, ref in enumerate(datasetRefs):
    all_ref_for_src.append(ref.dataId)
    print("========================datasetType = objectTable ============================================")
    print("fullId..................:",ref.dataId.full)
    print("skymap...................:",ref.dataId["skymap"])
    print("tract....................:",ref.dataId["tract"])
    print("patch....................:",ref.dataId["patch"])
    print("run......................:",ref.run)
    
    df_tab=  butler.get(ref,collections=my_collection)

    N=len(df_tab)
    df_tab['tabid_idx']= (i*np.ones(N)).astype(int)
    
    all_objectsTables.append(df_tab) 
    

In [ ]:
len(all_objectsTables)

In [ ]:
all_objectsTables[0]

In [ ]:
df=pd.concat(all_objectsTables)

In [ ]:
df

In [ ]:
df.to_csv(fileobj_out)

In [ ]:
#print(df.columns.tolist())

## List of patch and tract

In [ ]:
tracts_nums = df["tract"].unique()
tracts_nums

In [ ]:
tracts_patchs_dict = {}
for tract_num in tracts_nums:
    df_tract = df[df.tract == tract_num]
    patchs_nums = df_tract["patch"].unique()
    print(f"{tract_num} : ",patchs_nums)
    tracts_patchs_dict[tract_num] = patchs_nums 

In [ ]:
import pickle
# Store data (serialize)
with open(filetractpatch_out, 'wb') as handle:
    pickle.dump(tracts_patchs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Coordinates

In [ ]:
df_sel = df[['coord_ra','coord_dec','tract','patch']]

In [ ]:
df_sel

In [ ]:
df_sel.columns

In [ ]:
df_sel.plot.scatter(x='coord_ra',y='coord_dec',c='tract',colormap='tab20',grid=True,title="AUXTEL photometry 2023")

In [ ]:
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
ra = coord.Angle(df_sel["coord_ra"].values, unit=u.degree)
dec = coord.Angle(df_sel["coord_dec"].values, unit=u.degree)

In [ ]:
ra  = ra.wrap_at(180*u.degree).radian
dec = dec.radian

In [ ]:
df_sel["RA"] = ra
df_sel["DEC"] = dec

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="aitoff")
df_sel.plot.scatter(x='RA',y='DEC',c='tract',colormap='tab20',grid=True,ax=ax,figsize=(10,8),title='AUXTEL photometry 2023')